# Data Engineering Challenge

Import libraries

In [ ]:
import requests
import json
import pandas as pd
import sqlite3

## Data Ingestion

Obtain the JSON data from the GitHub repository

In [ ]:
url = ' https://raw.githubusercontent.com/mattcattaneo21/data-eng-challenge/refs/heads/main/mock_event_logs.json'
response = requests.get(url)
data = response.json()

print(str(len(data)) + ' rows were uploaded\n')

print('Sample of the first row:')
print(data[0])

500 rows were uploaded

Sample of the first row:
{'event_id': 'f3849d71-d832-4777-b097-e38dd89477a8', 'timestamp': '2025-05-31T14:36:20.971025', 'event_type': 'comment_added', 'user_id': 'user_1', 'document_id': 'doc_4', 'comment_text': 'Comment 46'}


Convert JSON file into a DataFrame

In [ ]:
df = pd.DataFrame(data)
df.head()

,event_id,timestamp,event_type,user_id,document_id,comment_text,shared_with,edit_length
0,f3849d71-d832-4777-b097-e38dd89477a8,2025-05-31T14:36:20.971025,comment_added,user_1,doc_4,Comment 46,NaN,NaN
1,37b91816-ac0f-45f2-923a-235c99dd7ddd,2025-06-04T19:35:09.971025,document_shared,user_9,doc_1,NaN,user_5,NaN
2,1752d4b4-2031-4654-a91c-a286fec1209a,2025-06-07T05:19:00.971025,user_login,user_1,doc_1,NaN,NaN,NaN
3,1ae41ba5-f302-4fe7-b7b5-a8819880da90,2025-05-31T10:22:06.971025,document_edit,user_4,doc_1,NaN,NaN,787.0
4,14c274fc-b403-4cff-baa1-daeacd839b2f,2025-05-19T21:31:38.971025,document_edit,user_10,doc_2,NaN,NaN,151.0


## Data Transformation

Divide the DataFrame into 3 normalized tables:

Users table

In [ ]:
# Users table without duplicates
users_df = pd.DataFrame(df['user_id'].unique(), columns=['user_id'])

# Check the quantity of Users
print(f'{len(users_df)} unique users\n')
users_df.head()

10 unique users



,user_id
0,user_1
1,user_9
2,user_4
3,user_10
4,user_6


Documents table

In [ ]:
# Documents table without duplicates
documents_df = pd.DataFrame(df['document_id'].unique(), columns=['document_id'])

# Check the quantity of Documents
print(f'{len(documents_df)} unique documents\n')
documents_df.head()

5 unique documents



,document_id
0,doc_4
1,doc_1
2,doc_2
3,doc_5
4,doc_3


Events table

In [ ]:
# Events table
# Make a copy of the original DataFrame to use all the fields
events_df = df.copy()

# Convert the 'timestamp' column to datetime
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
# Order the rows by timestamp from oldest to newest
events_df = events_df.sort_values(by='timestamp').reset_index(drop=True)

events_df.head()

,event_id,timestamp,event_type,user_id,document_id,comment_text,shared_with,edit_length
0,47f3543b-8da9-4ff9-ada4-3881d570f76c,2025-05-11 19:26:46.971025,document_shared,user_2,doc_4,NaN,user_5,NaN
1,572899d5-6e9d-4002-ab0d-b11886db96d3,2025-05-11 19:29:05.971025,document_edit,user_5,doc_5,NaN,NaN,665.0
2,c801b6d4-c948-42be-83d6-624fc44f4be6,2025-05-11 20:38:21.971025,document_shared,user_2,doc_1,NaN,user_4,NaN
3,209d307b-c877-46f2-9a93-45e243259f22,2025-05-11 21:31:28.971025,comment_added,user_2,doc_5,Comment 7,NaN,NaN
4,2a26fc12-9eb4-4878-bad1-b7a094214b45,2025-05-12 00:02:30.971025,user_login,user_7,doc_4,NaN,NaN,NaN


### Data cleansing and enrichment

In [ ]:
# Check if there are duplicate events and drop them
print(f'Total events: {len(events_df)}')

events_df = events_df.drop_duplicates(subset='event_id')
print(f'Number of events without duplicates: {len(events_df)}')

Total events: 500
Number of events without duplicates: 500


In [ ]:
#Events with empty values
malformed = events_df[events_df[['event_type', 'user_id', 'document_id']].isnull().any(axis=1)]
print(f'Events with issues: {len(malformed)}')

Events with issues: 0


In [ ]:
# Dropping malformed events
events_df = events_df.dropna(subset=['event_type', 'user_id', 'document_id'])

### Addition of derived columns

Day of week

In [ ]:
events_df['day_of_week'] = events_df['timestamp'].dt.day_name()
events_df[['timestamp', 'day_of_week']].head()

,timestamp,day_of_week
0,2025-05-11 19:26:46.971025,Sunday
1,2025-05-11 19:29:05.971025,Sunday
2,2025-05-11 20:38:21.971025,Sunday
3,2025-05-11 21:31:28.971025,Sunday
4,2025-05-12 00:02:30.971025,Monday


Session duration

In [ ]:
# Filter login events
logins = events_df[events_df['event_type'] == 'user_login'].copy()

# Order by user and timestamp
logins = logins.sort_values(['user_id', 'timestamp'])

# Calculate session duration
logins['session_duration'] = logins.groupby('user_id')['timestamp'].diff().shift(-1)

# Drop existing session_duration column in events_df (if it exists)
events_df = events_df.drop(columns=['session_duration'], errors='ignore')

# Merge the session_duration column back into events_df
events_df = events_df.merge(logins[['event_id', 'session_duration']], on='event_id', how='left')

# Convert session_duration to string
events_df['session_duration'] = events_df['session_duration'].astype(str)

# Check result
events_df[['event_id', 'session_duration']].head()

,event_id,session_duration
0,47f3543b-8da9-4ff9-ada4-3881d570f76c,NaT
1,572899d5-6e9d-4002-ab0d-b11886db96d3,NaT
2,c801b6d4-c948-42be-83d6-624fc44f4be6,NaT
3,209d307b-c877-46f2-9a93-45e243259f22,NaT
4,2a26fc12-9eb4-4878-bad1-b7a094214b45,0 days 03:06:44


Document word count

In [ ]:
# Document word count
def doc_word_count(edit_length):
    if pd.isna(edit_length):
        return 0
    else:
        return int(edit_length)

events_df['document_word_count'] = events_df['edit_length'].apply(doc_word_count)

# Sort by document_word_count descending, then take top 5 rows
events_df[['event_id','event_type','timestamp','document_id','document_word_count']].sort_values(by='document_word_count', ascending=False).head()

,event_id,event_type,timestamp,document_id,document_word_count
305,2c071b4e-623c-41c2-b094-50866a63244b,document_edit,2025-05-29 12:50:40.971025,doc_3,984
383,cd3539a4-16f4-48de-8260-55201c1d7db3,document_edit,2025-06-03 23:34:32.971025,doc_1,983
199,6c77be1b-44f3-4a88-ba0f-78f54f8c91df,document_edit,2025-05-23 08:13:38.971025,doc_4,976
491,e06d4703-4074-40c4-acf4-1b3f3ef8e65f,document_edit,2025-06-10 04:03:33.971025,doc_2,973
109,e7b11890-ff37-4b4b-b673-eb668233b282,document_edit,2025-05-17 10:05:47.971025,doc_3,967


Final table sample

In [ ]:
events_df.head()

,event_id,timestamp,event_type,user_id,document_id,comment_text,shared_with,edit_length,day_of_week,session_duration,document_word_count
0,47f3543b-8da9-4ff9-ada4-3881d570f76c,2025-05-11 19:26:46.971025,document_shared,user_2,doc_4,NaN,user_5,NaN,Sunday,NaT,0
1,572899d5-6e9d-4002-ab0d-b11886db96d3,2025-05-11 19:29:05.971025,document_edit,user_5,doc_5,NaN,NaN,665.0,Sunday,NaT,665
2,c801b6d4-c948-42be-83d6-624fc44f4be6,2025-05-11 20:38:21.971025,document_shared,user_2,doc_1,NaN,user_4,NaN,Sunday,NaT,0
3,209d307b-c877-46f2-9a93-45e243259f22,2025-05-11 21:31:28.971025,comment_added,user_2,doc_5,Comment 7,NaN,NaN,Sunday,NaT,0
4,2a26fc12-9eb4-4878-bad1-b7a094214b45,2025-05-12 00:02:30.971025,user_login,user_7,doc_4,NaN,NaN,NaN,Monday,0 days 03:06:44,0


## Data Storage

In [ ]:
# This creates a file-based SQLite database
conn = sqlite3.connect('events_data.db')

# Create the tables into the db
users_df.to_sql('users', conn, if_exists='replace', index=False)
documents_df.to_sql('documents', conn, if_exists='replace', index=False)
events_df.to_sql('events', conn, if_exists='replace', index=False)

print("'users', 'documents' and 'events' tables added to the 'events_data.db'")

'users', 'documents' and 'events' tables added to the 'events_data.db'


In [ ]:
# Checking that the users table is showing values correctly

# Connect to the database
conn = sqlite3.connect('events_data.db')

# Query to show the unique users
query = "SELECT * FROM users"
df_users = pd.read_sql_query(query, conn)

# Show the first 5 users
print(df_users.head())

   user_id
0   user_1
1   user_9
2   user_4
3  user_10
4   user_6


## Analytics

Daily Active Users over the last 30 days

In [ ]:
query = """
SELECT
    DATE(timestamp) as day,
    COUNT(DISTINCT user_id) as daily_active_users
FROM events
WHERE DATE(timestamp) >= DATE('now', '-30 day')
GROUP BY day
ORDER BY day DESC
"""
pd.read_sql_query(query, conn)

,day,daily_active_users
0,2025-06-10,7
1,2025-06-09,9
2,2025-06-08,9
3,2025-06-07,7


Average session duration by user (in seconds)

In [ ]:
# Convert session_duration to seconds directly (overwrite the column)
events_df['session_duration'] = pd.to_timedelta(events_df['session_duration']).dt.total_seconds()

# Save to SQLite (this will store session_duration as a float in seconds)
events_df.to_sql('events', conn, if_exists='replace', index=False)

500

In [ ]:
query = """
SELECT
    user_id,
    AVG(session_duration) AS avg_session_duration_seconds
FROM events
WHERE session_duration IS NOT NULL
GROUP BY user_id
ORDER BY avg_session_duration_seconds DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,user_id,avg_session_duration_seconds
0,user_6,244651.000000
1,user_10,240673.500000
2,user_8,224139.222222
3,user_3,222717.300000
4,user_9,221167.600000
5,user_4,194593.461538
6,user_5,178334.300000
7,user_1,178130.333333
8,user_2,167435.714286
9,user_7,150581.666667


Top 10 most edited documents

In [ ]:
query = """
SELECT
    document_id,
    COUNT(*) as edit_count
FROM events
WHERE event_type = 'document_edit'
GROUP BY document_id
ORDER BY edit_count DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)

,document_id,edit_count
0,doc_4,27
1,doc_2,25
2,doc_5,24
3,doc_1,22
4,doc_3,19


Number of shared documents per user

In [ ]:
query = """
SELECT
    user_id,
    COUNT(*) as shared_documents
FROM events
WHERE event_type = 'document_shared'
GROUP BY user_id
ORDER BY shared_documents DESC
"""
pd.read_sql_query(query, conn)


,user_id,shared_documents
0,user_7,21
1,user_9,18
2,user_6,18
3,user_10,15
4,user_8,12
5,user_5,10
6,user_3,10
7,user_4,9
8,user_2,8
9,user_1,8


In [ ]:


conn = sqlite3.connect("events_data.db")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(events);")
for col in cursor.fetchall():
    print(col)


(0, 'event_id', 'TEXT', 0, None, 0)
(1, 'timestamp', 'TIMESTAMP', 0, None, 0)
(2, 'event_type', 'TEXT', 0, None, 0)
(3, 'user_id', 'TEXT', 0, None, 0)
(4, 'document_id', 'TEXT', 0, None, 0)
(5, 'comment_text', 'TEXT', 0, None, 0)
(6, 'shared_with', 'TEXT', 0, None, 0)
(7, 'edit_length', 'REAL', 0, None, 0)
(8, 'day_of_week', 'TEXT', 0, None, 0)
(9, 'session_duration', 'REAL', 0, None, 0)
(10, 'document_word_count', 'INTEGER', 0, None, 0)
